# Reuters-21578 collection

Reuters-21578 is arguably the most commonly used collection for text classification during the last two decades.
It has been used in some of the most influential papers on the field. 

This dataset contains structured information about newswire articles that can be assigned to several classes, 
therefore making this a multi-label problem. It has a highly skewed distribution of documents over categories, 
where a large proportion of documents belong to few topics.

The collection originally consists of 21,578 documents, including documents without topics and typographical errors. 
For this reason, a subset and split of the collection is traditionally used. This split also focus only on the 
categories that have at least one document in the training set and the test set. The dataset has 90 categories with a 
training set of 7769 documents and a test set of 3019 documents.


# Data Collection Stats

Understanding your collection is always the first step of any data science problem. Lets have a quick look at the Reuters collection and its documents.

In [ ]:
from nltk.corpus import reuters

# List of document ids
documents = reuters.fileids()
print("Documents: {}".format(len(documents)))

# Train documents
train_docs_id = list(filter(lambda doc: doc.startswith("train"), documents))
print("Total train documents: {}".format(len(train_docs_id)))

# Test documents
test_docs_id = list(filter(lambda doc: doc.startswith("test"), documents))
print("Total test documents: {}".format(len(test_docs_id)))

In [ ]:
# Example of a document (with multiple labels)
doc = 'training/9865'
print(reuters.raw(doc))
print()

print(reuters.categories(doc))

In [ ]:
from operator import itemgetter
from pprint import pprint

# List of categories 
categories = reuters.categories();
print("Number of categories: {}".format(len(categories)))
print()

print(categories)
print()

# Documents per category.
category_distribution = [(category, len(reuters.fileids(category))) 
                         for category in categories]

category_distribution = sorted(category_distribution, 
                               key=itemgetter(1), 
                               reverse=True)

print("Most common categories")
pprint(category_distribution[:5])
print()

print("Least common categories")
pprint(category_distribution[-5:])
print()

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

stop_words = stopwords.words("english")

train_docs_id = list(filter(lambda doc: doc.startswith("train"), documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"), documents))

train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]

# Tokenisation 
vectorizer = TfidfVectorizer(stop_words=stop_words)

# Learn and transform train documents
vectorised_train_documents = vectorizer.fit_transform(train_docs)
vectorised_test_documents = vectorizer.transform(test_docs)

# Transform multilabel labels
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([reuters.categories(doc_id) 
                                  for doc_id in train_docs_id])
test_labels = mlb.transform([reuters.categories(doc_id) 
                             for doc_id in test_docs_id])

# Classifier 
classifier = OneVsRestClassifier(LinearSVC(random_state=42))
classifier.fit(vectorised_train_documents, train_labels)
predictions = classifier.predict(vectorised_test_documents)

print("Number of labels assigned: {}".format(sum([sum(prediction) 
                                                  for prediction in predictions])))

# How well have we done?

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Show our quality
precision = precision_score(test_labels, predictions, average='micro')
recall = recall_score(test_labels, predictions, average='micro')
f1 = f1_score(test_labels, predictions, average='micro')
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, 
                                                                     recall, 
                                                                     f1))

precision = precision_score(test_labels, predictions, average='macro')
recall = recall_score(test_labels, predictions, average='macro')
f1 = f1_score(test_labels, predictions, average='macro')
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, 
                                                                     recall, 
                                                                     f1))

## What is this weird warning message about "Precision being ill-defined?"

Some metrics can be in a position of indeterminate value when for instance, the classifier decides to not classify any articles in a specific category. This would imply a 0/0 precision. Scikit learn warns us about this, and reports back this quality as 0.0.